# A weighty problem
  - The maximum a crate can carry is $25kg$; this means the maximum three crates can carry is $25 \times 3 = 75kg$.
  - The total weight of boxes is $75kg$, which means each crate must carry exactly $25kg$.

With this information, the problem can now be redefined with the following statement:

How can I find three distinct combinations of boxes that weigh $25kg$


# Definition
\begin{equation}
\mathcal{U} = \{(i, w) \mid i \in \{0, \ldots, 9\}, w = W(i)\}
\end{equation}\
$W: \{0, \ldots, 9\} \rightarrow \mathbb{N}$ is defined by:
\begin{align}
W(0) &= 15, \quad W(1) = 13, \quad W(2) = 11, \quad W(3) = 10, \quad W(4) = 9, \\
W(5) &= 8, \quad W(6) = 4, \quad W(7) = 2, \quad W(8) = 2, \quad W(9) = 1
\end{align}\
Let $\mathcal{C} = \{C_1, C_2, C_3\}$ where each $C_k \subset \mathcal{U}$.\\
Find a partition $\mathcal{C}$ of $\mathcal{U}$ such that:

  - **Completeness**:
    \begin{equation}
    \bigcup_{k=1}^{3} C_k = \mathcal{U}
    \end{equation}
    
  - **Disjointness:**
    \begin{equation}
    C_i \cap C_j = \emptyset \quad \text{for all } i \neq j
    \end{equation}
    
  - **Weight constraint:**
    \begin{equation}
    W(C_k) = \sum_{(i,w) \in C_k}w = 25 \quad \text{for all } k \in \{1, 2, 3\}
    \end{equation}

# Formula
\begin{equation}
\varphi = \left(\bigwedge^3_{k=1}(W(C_k) = 25)\right)
         \wedge \left(\bigcup_{k=1}^3 C_k = \mathcal{U}\right)
         \wedge \left(\bigwedge_{1 \leq i < j \leq 3}(C_i \cap C_j = \emptyset)\right)
\end{equation}

In [112]:
from pysmt.shortcuts import Symbol, Int, Not, BV, And, Or, Equals, Plus, Solver, GE, LE, Ite, BVMul, AllDifferent, BVAdd, BVULE, BVUGE, Times
from pysmt.typing import INT, BVType

In [113]:

boxes = {0: 15, 1:13, 2:11, 3:10, 4:9, 5:8, 6:4, 7:2, 8:2, 9:1}
num_boxes = len(boxes)
num_crates = 3
target_weight = 25

box_crate = {}
for i in range(num_boxes):
    box_crate[i] = Symbol(f"box_{i}_crate", INT)

constraints = []

for i in range(num_boxes):
    constraints.append(GE(box_crate[i], Int(0)))
    constraints.append(LE(box_crate[i], Int(num_crates - 1)))

for k in range(num_crates):
    crate_weight_terms = []

    for i in range(num_boxes):
        crate_weight_terms.append(
            Ite(Equals(box_crate[i], Int(k)),
                Int(boxes[i]),
                Int(0))
        )
    crate_total = Plus(crate_weight_terms)
    constraints.append(Equals(crate_total, Int(target_weight)))

formula = And(constraints)

with Solver(name="z3") as solver:
    solver.add_assertion(formula)

    if solver.solve():
        print(" Solution found !\n")
        model = solver.get_model()
        crates = {0: [], 1: [], 2: []}
        
        for i in range(num_boxes):
            crate_num = model.get_value(box_crate[i]).constant_value()
            crates[crate_num].append(i)
        
        for k in range(num_crates):
            box_list = crates[k]
            weights = [boxes[i] for i in box_list]
            total = sum(weights)
            
            print(f"Crate {k + 1}:")
            print(f"  Boxes: {box_list}")
            print(f"  Weights: {weights}")
            print(f"  Total weight: {total}kg")

 Solution found !

Crate 1:
  Boxes: [3, 4, 6, 7]
  Weights: [10, 9, 4, 2]
  Total weight: 25kg
Crate 2:
  Boxes: [1, 2, 9]
  Weights: [13, 11, 1]
  Total weight: 25kg
Crate 3:
  Boxes: [0, 5, 8]
  Weights: [15, 8, 2]
  Total weight: 25kg


# Algebra Logic

- Solve the following (put digits for the letters):
  \begin{equation}
        (L + O + G + I + C)^3 = L*10000 + O*1000 + G*100 + I*10 
    \end{equation}
  \begin{align}
      \Omega = \{L, O, G, I, C\}\ \subset \mathbb{N};\\
      \forall\; l \in \Omega \;, 0  \leq l \leq 9
  \end{align}

  Base on this definition below, the problem can be expressed with the following formula:
  # Formula
  \begin{equation}
\varphi = \left(\        (L + O + G + I + C)^3 = L*10000 + O*1000 + G*100 + I*10 + C\right)
         \wedge \forall l \left((l\leq 9) \wedge (l \geq 0)\right)
\end{equation}

In [114]:
def BVConst(value):
    return BV(value, 16)

In [115]:
BV16 = BVType(16)

L = Symbol("L", BV16)
O = Symbol("O", BV16)
G = Symbol("G", BV16)
I = Symbol("I", BV16)
C = Symbol("C", BV16)

variables = [L, O, G, I, C]

constraints = []

for var in variables:
    constraints.append(BVUGE(var, BVConst(0)))
    constraints.append(BVULE(var, BVConst(9)))

constraints.append(Not(Equals(L, BVConst(0))))

#constraints.append(AllDifferent(variables))

sum_vars = BVAdd(BVAdd(BVAdd(BVAdd(L, O), G), I), C)
sum_squared = BVMul(sum_vars, sum_vars)
left_side = BVMul(sum_squared, sum_vars)

right_side = BVAdd(
    BVAdd(
        BVAdd(
            BVAdd(
                BVMul(L, BVConst(10000)),
                BVMul(O, BVConst(1000))
            ),
            BVMul(G, BVConst(100))
        ),
        BVMul(I, BVConst(10))
    ),
    C
)

constraints.append(Equals(left_side, right_side))

formula = And(constraints)

with Solver(name="z3") as solver:
    solver.add_assertion(formula)

    if solver.solve():
        model = solver.get_model()

        print("Solution found!")
        L_val = model.get_value(L).constant_value()
        O_val = model.get_value(O).constant_value()
        G_val = model.get_value(G).constant_value()
        I_val = model.get_value(I).constant_value()
        C_val = model.get_value(C).constant_value()
        
        print(f"L = {L_val}")
        print(f"O = {O_val}")
        print(f"G = {G_val}")
        print(f"I = {I_val}")
        print(f"C = {C_val}")
        print()
    else:
        print("No solution found!")

Solution found!
L = 1
O = 9
G = 6
I = 8
C = 3



# 10-digit Number
## Problem Statement

Find a 10-digit number where the first digit is how many zeros in the number, the second digit is how many 1s in the number etc. until the tenth digit is how many 9s is in the number.

## Resolution
The problem is to find a 10-digit number ($X = X_0X_1..X_i..X_9$) such that:
  -  $X_0 \rightarrow$ number of $0$ in $X_0X_1..X_i..X_9$
  -  $X_1 \rightarrow$ number of $1$ in  $X_0X_1..X_i..X_9$
  -  ... ... ...
  - $X_i \rightarrow$ number of $i$ in $X_0X_1..X_i..X_9$

Base on this we can make the following definition:
### Definition
$  \Omega = \left\{X_0, X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, |\; 0\leq X \leq 9 \right\} \subset  \mathbb{N}$

$$
  I:
  \begin{array}{rcl}
    \Omega & \longrightarrow & \mathbb{N} \\
    X_i & \longmapsto & i
  \end{array}
  \quad \text{with } i \; \text{the index of the digit } X_i \text{ in } X
$$
$$
  C_X:
  \begin{array}{rcl}
    \mathbb{N}\times \mathbb{N} & \longrightarrow & \mathbb{N}\\
    (X, i) & \longmapsto & \mathrm{Count_X}(i)
  \end{array}
  \quad \text{with } \mathrm{Count}_X(i) = \text{number of occurrences of } i \text{ in } X.
$$


$X_i \rightarrow$ number of $i$ in $X \equiv C_X(X,I(X_i)) = X_i$

Considering the definition below, we can now rewrite the problem with the following formula:
## Formula
\begin{equation}
\varphi = \left(\bigwedge^9_{i=0}((X_i \geq 0)\wedge(X_i \leq 9))\right)
         \wedge \left(\bigwedge_{i=0}^9 C_X(X, I(X_i)) = X_i\right)
\end{equation}

In [116]:
X_0 = Symbol("X_0", INT)
X_1 = Symbol("X_1", INT)
X_2 = Symbol("X_2", INT)
X_3 = Symbol("X_3", INT)
X_4 = Symbol("X_4", INT)
X_5 = Symbol("X_5", INT)
X_6 = Symbol("X_6", INT)
X_7 = Symbol("X_7", INT)
X_8 = Symbol("X_8", INT)
X_9 = Symbol("X_9", INT)

variables = [X_0, X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9]

constraints = []

for var in variables:
    constraints.append(GE(var, Int(0)))
    constraints.append(LE(var, Int(9)))

for i, var in enumerate(variables):
    count = Plus([Ite(Equals(v, Int(i)), Int(1), Int(0))
                for v in variables])
    constraints.append(Equals(var, count))
formula = And(constraints)

with Solver(name="z3") as solver:
    solver.add_assertion(formula)

    if solver.solve():
        model = solver.get_model()

        print("Solution found!")
        for i, var in enumerate(variables):
            val = model.get_value(var).constant_value()
            print(f"X_{i} = {val}")
    else:
        print("No solution found!")

Solution found!
X_0 = 6
X_1 = 2
X_2 = 1
X_3 = 0
X_4 = 0
X_5 = 0
X_6 = 1
X_7 = 0
X_8 = 0
X_9 = 0


# $\alpha + \beta + \gamma = \delta$
## Problem Statement

Solve this:
  - ALFA + BETA + GAMMA = DELTA
Replace letters with digits and have the sum be true. (There is more than one solution.)
### Resolution
The problem is to find digits satisfied this:

 $$(A\times 1000 + L\times 100 + F\times 10 + A)$$
  +
  $$(B\times 1000 + E\times 100 + T\times 10 + A)$$
  +
  $$(G\times 10000 + A\times 1000 + M\times 100 + M\times 10 + A)$$
  $$= D\times 10000 + E\times 1000 + L\times 100 + T\times 10 + A
$$
### Definition
Let us define the domaine $\Omega$ first: 
$\Omega = \{ A, B, D, E, F, G, L, M, T  |\; 0 \leq L \leq 9\} \subset \mathbb{N}$

$\alpha = A\times 1000 + L\times 100 + F\times 10 + A)$

$\beta = B\times 1000 + E\times 100 + T\times 10 + A)$

$\gamma =  (G\times 10000 + A\times 1000 + M\times 100 + M\times 10 + A)$

$\delta = D\times 10000 + E\times 1000 + L\times 100 + T\times 10 + A$

The problem can be expressed with the following formula:
## Formula
$$
  \varphi = \left(\bigwedge^9_{i=0}((0 \leq L_i)\wedge(L_i \leq 9))\right)\wedge \left( \alpha + \beta + \gamma = \delta \right)
$$

In [117]:
A = Symbol("A", INT)
B = Symbol("B", INT)
D = Symbol("D", INT)
E = Symbol("E", INT)
F = Symbol("F", INT)
G_1 = Symbol("G_1", INT)
L_1 = Symbol("L_1", INT)
M = Symbol("M", INT)
T = Symbol("T", INT)

variables = [A, B, D, E, F, G_1, L_1, M, T]

constraints_t = []

for var in variables:
    constraints_t.append(GE(var, Int(1)))
    constraints_t.append(LE(var, Int(9)))

alpha = A*1000 + L_1*100 + F*10 + A
beta = B*1000 + E*100 + T*10 + A
gamma = G_1*10000 + A*1000 + M*100 + M*10 + A
delta = D*10000 + E*1000 + L_1*100 + T*10 + A
left_sum = alpha + beta + gamma

constraints_t.append(Equals(left_sum, delta))
formula_t = And(constraints_t)

with Solver(name="z3") as solver:
    solver.add_assertion(formula_t)
    if solver.solve():
        model = solver.get_model()
        print("Solution trouvée!")
        for var in variables:
            print(f"{var} = {model.get_value(var)}")
    else:
        print("Aucune solution!")


Solution trouvée!
A = 5
B = 1
D = 9
E = 2
F = 2
G_1 = 8
L_1 = 1
M = 7
T = 1


# 9-digit number
## Problem Statement

There is a 9 digit number. No digit are repeated and rightmost digit is divisible by 1 and right 2 digits is divisble by 2, right 3 digits is divisible by 3 and so on, finally the whole number is divisible by 9.

## Resolution
- $X = X_0X_1X_2X_3X_4X_5X_6X_7X_8$
- $ \exists k_8 \;| X_8 = 1 \times k_8. \text{ obviously } k_8 = X_8$
- $\exists k_7 \;| X_7X_8 = 2 \times k_7 $
- $\exists k_6 \;| X_6X_7X_8 = 3 \times k_6$
- $\exists k_5 \;| X_5X_6X_7X_8 = 4 \times k_5$
- $\exists k_4 \;| X_4X_5X_6X_7X_8 = 5 \times k_4$
- $\exists k_3 \;| X_3X_4X_5X_6X_7X_8 = 6 \times k_3$
- $\exists k_2 \; | X_2X_3X_4X_5X_6X_7X_8 = 7 \times k_2$
- $\exists k_1 \; | X_1X_2X_3X_4X_5X_6X_7X_8 = 8 \times k_1$
- $\exists k_0 \;| (X = X_0X_1X_2X_3X_4X_5X_6X_7X_8) = 9 \times k_0$

### Definition
\begin{equation}
  \Omega = \{X_0, X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_7, X_8\}
\end{equation}

\begin{equation}
    \forall i, \forall j, i \neq j \rightarrow X_i \neq X_j
\end{equation}
\begin{equation}
  \forall i, 0\leq X_i \leq 9
\end{equation}

$$
  \sigma_X:
  \begin{array}{rcl}
    \Omega\times \mathbb{N} & \longrightarrow & \mathbb{N}\\
    (X, i) & \longmapsto & X_{i}..X_8
  \end{array}
  \quad \text{with } \sigma_X(X,8) = X_8 \text{ and } \sigma_X(X,0) = X
$$


The problem can be expressed with the following formula:

### Formula
\begin{equation}
  \varphi = \left(\bigwedge^8_{i=0}((0 \leq X_i)\wedge(X_i \leq 9))\right) \wedge \left(\bigwedge_{0 \leq i < j \leq 8}(X_i \neq X_j)\right) \wedge \left(\bigwedge_{0 \leq i \leq 8}(\sigma_X(X,i) = (9-i) \times k_i)\right)
\end{equation}

In [118]:
X0 = Symbol("X0", INT)
X1 = Symbol("X1", INT)
X2 = Symbol("X2", INT)
X3 = Symbol("X3", INT)
X4 = Symbol("X4", INT)
X5 = Symbol("X5", INT)
X6 = Symbol("X6", INT)
X7 = Symbol("X7", INT)
X8 = Symbol("X8", INT)

k0 = Symbol("k0", INT)
k1 = Symbol("k1", INT)
k2 = Symbol("k2", INT)
k3 = Symbol("k3", INT)
k4 = Symbol("k4", INT)
k5 = Symbol("k5", INT)
k6 = Symbol("k6", INT)
k7 = Symbol("k7", INT)
k8 = Symbol("k8", INT)

variables = [X0, X1, X2, X3, X4, X5, X6, X7, X8]

k_variables = [k0, k1, k2, k3, k4, k5, k6, k7, k8]

constraints = []

for var in variables:
    constraints.append(GE(var, Int(0)))
    constraints.append(LE(var, Int(9)))

constraints.append(AllDifferent(variables))

sigma_8 = X8
constraints.append(Equals(sigma_8, Times(Int(1), k8)))

sigma_7 = Plus(Times(Int(10), X7), sigma_8)
constraints.append(Equals(sigma_7, Times(Int(2), k7)))

sigma_6 = Plus(Times(Int(100), X6), sigma_7)
constraints.append(Equals(sigma_6, Times(Int(3), k6)))

sigma_5 = Plus(Times(Int(1000), X5), sigma_6)
constraints.append(Equals(sigma_5, Times(Int(4), k5)))

sigma_4 = Plus(Times(Int(10000), X4), sigma_5)
constraints.append(Equals(sigma_4, Times(Int(5), k4)))

sigma_3 = Plus(Times(Int(100000), X3), sigma_4)
constraints.append(Equals(sigma_3, Times(Int(6), k3)))

sigma_2 = Plus(Times(Int(1000000), X2), sigma_3)
constraints.append(Equals(sigma_2, Times(Int(7), k2)))

sigma_1 = Plus(Times(Int(10000000), X1), sigma_2)
constraints.append(Equals(sigma_1, Times(Int(8), k1)))

sigma_0 = Plus(Times(Int(100000000), X0), sigma_1)
constraints.append(Equals(sigma_0, Times(Int(9), k0)))

formula = And(constraints)

with Solver(name="z3") as solver:
    solver.add_assertion(formula)
    if solver.solve():
        model = solver.get_model()
        print("Solution found!")
        for var in variables:
            print(f"{var} = {model.get_value(var)}")
    else:
        print("No Solution found!")

Solution found!
X0 = 6
X1 = 4
X2 = 8
X3 = 3
X4 = 1
X5 = 5
X6 = 7
X7 = 2
X8 = 0
